In [1]:
from models.architecture.rnn_net import RnnPredictCandles
from tools.dataset import Dataset
from models.processing.create_features import createFeatures
from models.processing.processing_data import Transformer

import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
dataset = Dataset().get_data(days=30, ticker='BTCUSDT', ts='1h')

In [3]:
features = createFeatures().create_features(dataset[['open', 'high', 'low', 'close']])
cols = features.columns
transformer = Transformer()
X, y, indexes = transformer.transform_to_lstm_input(
    data=features,
    cols=cols,
    window_size=24,
    step_size=0,
    ahead=12
)

0it [00:00, ?it/s]

[X] Ooops!!! It's True, the series 'volume' was not found in open, high, low, close


/home/serg/projects/onramp/quant-ai/models/processing/create_features.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ta.strategy("Momentum")
39it [00:00, 84.05it/s]


In [4]:
X_split = transformer.split_data(X, 0.1)
y_split = transformer.split_data(y, 0.1)
indexes = transformer.split_data(indexes, 0.1)

model = RnnPredictCandles(
    input_shape=X.shape[1:],
    output_shape=y.shape[1:],
    epochs=1000
)
model.verbose = 1
model.batch_size = 128
model.build_model()
model.fit_train_test(X_split[0], y_split[0], X_split[1], y_split[1])


2022-09-01 16:11:58.680652: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-01 16:11:58.709300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/serg/.mujoco/mujoco210/bin:/home/serg/.mujoco/mujoco210/bin
2022-09-01 16:11:58.709994: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-09-01 16:11:58.710558: I tensorflow/core/platform/cpu_feature_guar

In [5]:
predicted = model.predict(X_split[1])
predicted = transformer.inverse_min_max_scaler_y(predicted[-1])
print(f'predicted: {predicted}')

38/38 [==============================] - 1s 12ms/step
predicted: [[20596.568 20757.158 20518.736 20600.186]
 [20607.928 20665.762 20465.025 20545.04 ]
 [20553.504 20655.504 20454.492 20537.393]
 [20552.062 20684.535 20484.275 20602.447]
 [20592.393 20718.482 20526.418 20618.324]
 [20636.785 20730.066 20530.365 20653.418]
 [20640.666 20761.467 20582.064 20623.11 ]
 [20623.344 20719.82  20558.115 20615.715]
 [20647.559 20745.408 20560.148 20605.754]
 [20637.965 20725.016 20524.38  20615.967]
 [20593.236 20719.57  20489.176 20609.223]
 [20586.35  20716.348 20444.37  20546.781]]


# Predict real time for the next 12 Hours (candles)

In [6]:
expected_prices = pd.DataFrame(predicted, columns=['open', 'high', 'low', 'close'])
fig = go.Figure(data=[go.Candlestick(
                x=expected_prices.index,
                open=expected_prices['open'],
                high=expected_prices['high'],
                low=expected_prices['low'],
                close=expected_prices['close'])])
fig.show()

In [8]:
indexes[1]

Timestamp('2022-08-07 02:00:00')

In [9]:
model.verbose

1